In [ ]:
from model import get_tb_model

In [ ]:
model_config = {
    "start_time": 1850,
    "end_time": 2050,
    "seed": 100,   
}

In [ ]:
studies_dict = {
    "study_1": {
        "pop_size": 2.e6,
    },
    "study_2": {
        "pop_size": 1.e6,
    }    
}

In [ ]:
model = get_tb_model(model_config, studies_dict)

In [ ]:
model.compartments

In [ ]:
model.get_input_parameters()